<a href="https://colab.research.google.com/github/kaledai069/Crossword-Generator/blob/master/Clues_and_Answer_setter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/kaledai069/Crossword-Solver-A-Neural-Transformer-based-Approach
%cd Crossword-Solver-A-Neural-Transformer-based-Approach/
!bash inference_setup.sh

Cloning into 'Crossword-Solver-A-Neural-Transformer-based-Approach'...
remote: Enumerating objects: 1817, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (216/216), done.
remote: Total 1817 (delta 328), reused 488 (delta 299), pack-reused 1300
Receiving objects: 100% (1817/1817), 12.00 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (522/522), done.
/content/Crossword-Solver-A-Neural-Transformer-based-Approach
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import time
import json
import random

from tqdm import tqdm
from pprint import pprint
from utils import puz_to_json, fetch_nyt_crossword
from solver.Crossword import Crossword

In [25]:
curr_date = datetime.date(2023, 12, 30)
formatted_date = curr_date.strftime('%m/%d/%Y')
crossword_json_data = fetch_nyt_crossword(formatted_date)
crossword_json_data

{'metadata': {'date': '12/30/2023', 'rows': 15, 'cols': 15},
 'clues': {'across': {'1': ["Dad on 'Black-ish'", 'DRE'],
   '4': ['Source of intelligence', 'RECON'],
   '8': ['They work in meters', 'POETS'],
   '12': ['Bucket of bolts', 'JUNKER'],
   '14': ['Concealer', 'VEIL'],
   '16': ["Hour at which to sing 'Auld Lang Syne'", 'XII'],
   '17': ['It might have a small bulb', 'MEDICINEDROPPER'],
   '20': ['Coat named for a former Irish province', 'ULSTER'],
   '21': ['Brand with a joystick', 'ATARI'],
   '22': ['Letter of completion, in brief', 'CERT'],
   '24': ['Post-op program', 'REHAB'],
   '26': ['Took a turn', 'WENT'],
   '28': ["'Count' in the Blues Hall of Fame", 'BASIE'],
   '30': ['Uncertain affirmation', 'IGUESSSO'],
   '32': ['Boot', 'CAN'],
   '33': ['Pair of hand drums, in Indian music', 'TABLA'],
   '35': ['Speakers in many classrooms, for short', 'PAS'],
   '36': ['Rarer than rare', 'ONCEINALIFETIME'],
   '40': ['Directional suffix', 'ERN'],
   '41': ['Craft at camp', 'C

In [109]:
def find_grid_position(grid_nums, clue_num, n_rows):
    reshaped_grid = [grid_nums[i:i + n_rows] for i in range(0, len(grid_nums), n_rows)]

    for i in range(len(reshaped_grid)):
      for j in range(len(reshaped_grid[0])):
        if reshaped_grid[i][j] == clue_num:
          return (i, j)

def find_answer_length(main_grid, position, grid_size, across = True):
  rows, cols = grid_size
  i, j = position
  count = 0
  if across:
    for idx in range(j, cols):
      if main_grid[i][idx] == '.':
        return count
      else:
        count += 1
  else:
    for idx in range(i, rows):
      if main_grid[idx][j] == '.':
        return count
      else:
        count += 1
  return count

In [116]:
def json_CA_json_converter(json_file_path, is_path):
    if is_path:
        with open(json_file_path, "r") as file:
            data = json.load(file)
    else:
        data = json_file_path

    json_conversion_dict = {}

    rows = data["size"]["rows"]
    cols = data["size"]["cols"]

    grid_list = data['grid']

    main_grid = [grid_list[i:i + rows] for i in range(0, len(grid_list), rows)]


    json_conversion_dict["metadata"] = {"rows": rows, "cols": cols}

    across_grid_nums = data['across_nums']
    down_grid_nums = data['down_nums']

    across_clue_answer = {}
    down_clue_answer = {}

    grid_info = data["grid"]
    grid_num = data["gridnums"]
    reshaped_grid = [grid_num[i:i + rows] for i in range(0, len(grid_num), rows)]

    for clue_num in across_grid_nums:
      position = find_grid_position(grid_num, clue_num, rows)
      answer_count = find_answer_length(main_grid, position, (rows, cols))
      across_clue_answer[str(clue_num)] = ['Clue', "".join(['A' for _ in range(answer_count)])]

    for clue_num in down_grid_nums:
      position = find_grid_position(grid_num, clue_num, rows)
      answer_count = find_answer_length(main_grid, position, (rows, cols), False)
      down_clue_answer[str(clue_num)] = ['Clue', "".join(['A' for _ in range(answer_count)])]

    json_conversion_dict["clues"] = {
          "across": across_clue_answer,
          "down": down_clue_answer,
      }

    grid_info_list = []
    for i in range(rows):
        row_list = []
        for j in range(cols):
            if grid_info[i * rows + j] == ".":
                row_list.append("BLACK")
            else:
                if grid_num[i * rows + j] == 0:
                    row_list.append(["", grid_info[i * rows + j]])
                else:
                    row_list.append(
                        [str(grid_num[i * rows + j]), grid_info[i * rows + j]]
                    )
        grid_info_list.append(row_list)

    json_conversion_dict["grid"] = grid_info_list

    return json_conversion_dict

In [117]:
json_puzzle_path = "/content/crossword_json_7x7.json"
puzzle = json_CA_json_converter(json_puzzle_path, True)

puzzle

{'metadata': {'rows': 7, 'cols': 7},
 'clues': {'across': {'1': ['Clue', 'AAAA'],
   '5': ['Clue', 'AAAAAA'],
   '7': ['Clue', 'AAAAAAA'],
   '8': ['Clue', 'AAA'],
   '9': ['Clue', 'AAA'],
   '10': ['Clue', 'AAAAAAA'],
   '12': ['Clue', 'AAAAAA'],
   '13': ['Clue', 'AAAA']},
  'down': {'1': ['Clue', 'AAA'],
   '2': ['Clue', 'AAAAAA'],
   '3': ['Clue', 'AAAAAA'],
   '4': ['Clue', 'AAAAA'],
   '5': ['Clue', 'AAAAAA'],
   '6': ['Clue', 'AAAAAA'],
   '7': ['Clue', 'AAAAA'],
   '11': ['Clue', 'AAA']}},
 'grid': [['BLACK',
   'BLACK',
   'BLACK',
   ['1', ' '],
   ['2', ' '],
   ['3', ' '],
   ['4', ' ']],
  ['BLACK',
   ['5', ' '],
   ['6', ' '],
   ['', ' '],
   ['', ' '],
   ['', ' '],
   ['', ' ']],
  [['7', ' '],
   ['', ' '],
   ['', ' '],
   ['', ' '],
   ['', ' '],
   ['', ' '],
   ['', ' ']],
  [['8', ' '],
   ['', ' '],
   ['', ' '],
   'BLACK',
   ['9', ' '],
   ['', ' '],
   ['', ' ']],
  [['10', ' '],
   ['', ' '],
   ['', ' '],
   ['11', ' '],
   ['', ' '],
   ['', ' '],
   [''

In [48]:
def clean(text):
    return " ".join(text.strip().split())

class Crossword:
    def __init__(self, data):
        self.initialize_grids(grid = data["grid"]) #okay i have this
        self.initialize_clues(clues = data["clues"]) #have got no clue at all (literally)
        self.initialize_variables()

    def initialize_grids(self, grid):
        self.letter_grid = [[grid[j][i][1] if type(grid[j][i]) == list else "" for i in
                             range(len(grid[0]))] for j in range(len(grid))]
        self.number_grid = [[grid[j][i][0] if type(grid[j][i]) == list else "" for i in
                             range(len(grid[0]))] for j in range(len(grid))]
        self.grid_cells = {}

    def initialize_clues(self, clues):
        self.across = clues["across"]
        self.down = clues["down"]

    def initialize_variable(self, position, clues, across=True):
        row, col = position
        cell_number = self.number_grid[row][col]
        assert cell_number in clues, print("Missing clue")
        word_id = cell_number + "A" if across else cell_number + "D"
        clue = clean(clues[cell_number][0])
        answer = clean(clues[cell_number][1])
        for idx in range(len(answer)):
            cell = (row, col + idx) if across else (row + idx, col)
            if cell in self.grid_cells:
                self.grid_cells[cell].append(word_id)
            else:
                self.grid_cells[cell] = [word_id]
            if word_id in self.variables:
                self.variables[word_id]["cells"].append(cell)
            else:
                self.variables[word_id] = {"clue": clue, "gold": answer, "cells": [cell], "crossing": []}

    def initialize_crossing(self):
        for word_id in self.variables:
            cells = self.variables[word_id]["cells"]
            crossing_ids = []
            for cell in cells:
                crossing_ids += list(filter(lambda x: x!= word_id, self.grid_cells[cell]))
            self.variables[word_id]["crossing"] = crossing_ids

    def initialize_variables(self):
        self.variables = {}
        for row in range(len(self.number_grid)):
            for col in range(len(self.number_grid[0])):
                cell_number = self.number_grid[row][col]
                if self.number_grid[row][col] != "":
                    if cell_number in self.across:
                        self.initialize_variable((row, col), self.across, across=True)
                    if cell_number in self.down:
                        self.initialize_variable((row, col), self.down, across=False)
        self.initialize_crossing()

In [118]:
crossword = Crossword(puzzle)

In [120]:
UNIGRAM_PROBS = [('A', 0.0897379968935765), ('B', 0.02121248877769636), ('C', 0.03482206634145926), ('D', 0.03700942543460491), ('E', 0.1159773210750429), ('F', 0.017257461694024614), ('G', 0.025429024796296124), ('H', 0.033122967601502), ('I', 0.06800036223479956), ('J', 0.00294611331754349), ('K', 0.013860682888259786), ('L', 0.05130800574373874), ('M', 0.027962776827660175), ('N', 0.06631994270448001), ('O', 0.07374646543246745), ('P', 0.026750756212433214), ('Q', 0.001507814175439393), ('R', 0.07080460813737305), ('S', 0.07410988246048224), ('T', 0.07242993582154593), ('U', 0.0289272388037645), ('V', 0.009153522059555467), ('W', 0.01434705167591524), ('X', 0.003096729223103298), ('Y', 0.01749958208224007), ('Z', 0.002659777584995724)]
LETTER_SMOOTHING_FACTOR = [0.0, 0.0, 0.04395604395604396, 0.0001372495196266813, 0.0005752186417796561, 0.0019841824329989103, 0.0048042463338563764, 0.013325257419745608, 0.027154447774285505, 0.06513517299341645, 0.12527790128946198, 0.22003002358996354, 0.23172376584839494, 0.254873006497342, 0.3985086992543496, 0.2764976958525346, 0.672645739910314, 0.6818181818181818, 0.8571428571428571, 0.8245614035087719, 0.8, 0.71900826446281, 0.0]

In [122]:
class BPVar:
    def __init__(self, name, variable, candidates, cells):
        self.name = name # name of the filling, which goes by 1A, 2A, 1D and such
        cells_by_position = {}
        for cell in cells:
            cells_by_position[cell.position] = cell # this line is for (0, 0) -> 1A (BPCell)
            cell._connect(self) # this line is for connecting which BPCell is connected to which BPVariable

        self.length = len(cells) # length of the answer for that particular filling or variable
        self.ordered_cells = [cells_by_position[pos] for pos in variable['cells']] #[(0, 0), (0, 1), (0, 2)] -> [BPCell(1A), BPCell(2A), BPCell(3A)]{Ordered Cells}
        self.candidates = candidates # [words, bit_array, weights] for each filling produced by the First Pass Model
        self.words = self.candidates['words'] # all the possible answers that could fit in that particular across or down clue (less than the maximum candidates )

        self.word_indices = np.array([[string.ascii_uppercase.index(l) for l in fill] for fill in self.words]) # words x length of letter indices
        self.scores = -np.array([self.candidates['weights'][fill] for fill in self.candidates['words']]) # the incoming 'weights' are costs
        self.prior_log_probs = log_softmax(self.scores)
        self.log_probs = log_softmax(self.scores)
        self.directional_scores = [np.zeros(len(self.log_probs)) for _ in range(len(self.ordered_cells))]

    def _propagate_to_var(self, other, belief_state):
        assert other in self.ordered_cells
        other_idx = self.ordered_cells.index(other)
        letter_scores = belief_state
        self.directional_scores[other_idx] = letter_scores[self.word_indices[:, other_idx]]

    def _postprocess(self, all_letter_probs):
        # unigram smoothing
        unigram_probs = np.array([x[1] for x in UNIGRAM_PROBS])
        for i in range(len(all_letter_probs)):
            all_letter_probs[i] = (1 - LETTER_SMOOTHING_FACTOR[self.length]) * all_letter_probs[i] + LETTER_SMOOTHING_FACTOR[self.length] * unigram_probs
        return all_letter_probs

    def sync_state(self):
        self.log_probs = log_softmax(sum(self.directional_scores) + self.prior_log_probs)

    def propagate(self): # very first task in the main loop - applied across all the variables for that particular crossword
        all_letter_probs = []
        for i in range(len(self.ordered_cells)):
            word_scores = self.log_probs - self.directional_scores[i]
            word_probs = softmax(word_scores)
            letter_probs = (self.candidates['bit_array'][:, i] * np.expand_dims(word_probs, axis=0)).sum(axis=1) + 1e-8
            all_letter_probs.append(letter_probs)
        all_letter_probs = self._postprocess(all_letter_probs) # unigram postprocessing
        for i, cell in enumerate(self.ordered_cells):
            cell._propagate_to_cell(self, np.log(all_letter_probs[i]))

class BPCell:
    def __init__(self, position, clue_pair):
        self.crossing_clues = clue_pair
        self.position = tuple(position)
        self.letters = list(string.ascii_uppercase)
        self.log_probs = np.log(np.array([1./len(self.letters) for _ in range(len(self.letters))]))
        self.crossing_vars = []
        self.directional_scores = []
        self.prediction = {}

    def _connect(self, other):
        self.crossing_vars.append(other)
        self.directional_scores.append(None)
        assert len(self.crossing_vars) <= 2

    def _propagate_to_cell(self, other, belief_state):
        assert other in self.crossing_vars
        other_idx = self.crossing_vars.index(other)
        self.directional_scores[other_idx] = belief_state

    def sync_state(self):
        self.log_probs = log_softmax(sum(self.directional_scores))

    def propagate(self):
        # assert len(self.crossing_vars) == 2
        try:
            for i, v in enumerate(self.crossing_vars):
                v._propagate_to_var(self, self.directional_scores[1-i])
        except IndexError:
            pass

In [126]:
class Solver:
    def __init__(self, crossword, max_candidates = 1000, process_id=0):
        self.crossword = crossword
        self.max_candidates = max_candidates
        self.process_id = process_id
        self.get_candidates()
        self.clue_answer_dataset = "/content/gdrive/MyDrive/Clue-Answer Dataset/CLUE_ANSWER_DATA_VERSION_1.csv"
        self.load_dataset()

    def load_dataset(self):
        self.main_df = pd.read_csv(self.clue_answer_dataset)
        self.main_df = self.main_df[['clue', 'answer']]
        self.main_df['answer'] = self.main_df['answer'].apply(lambda x: x.replace(" ", '').upper() if not isinstance(x, float) else '-PRAJESH-INVALID-')
        self.main_df = self.main_df[~(self.main_df['answer'] == '-PRAJESH-INVALID-')]

    def generate_random_answers(self, answer_length_info, max_answers):
        for ans_len in answer_length_info:


    def get_candidates(self):
        # get answers from neural model and fill up data structures with the results
        chars = string.ascii_uppercase
        self.char_map = {char: idx for idx, char in enumerate(chars)}
        self.candidates = {}

        answer_length_info = []
        all_clues = []
        for var in self.crossword.variables:
            all_clues.append(self.crossword.variables[var]['clue'])
            answer_length_info.append(len(self.crossword.variables[var]['clue']['gold']))

        # get predictions
        all_words, all_scores = self.generate_random_answers(answer_length_info, max_answers = self.max_candidates)

        exit(0)
        for index, var in enumerate(self.crossword.variables):
            length = len(self.crossword.variables[var]["gold"])
            self.candidates[var] = {"words": [], "bit_array": None, "weights": {}}

            clue = all_clues[index]
            words, scores = all_words[index], all_scores[index]
            # remove answers that are not of the correct length
            keep_positions = []
            for word_index, word in enumerate(words):
                if len(word) == length:
                    keep_positions.append(word_index)
            words = [words[i] for i in keep_positions]
            scores = [scores[i] for i in keep_positions]
            scores = list(-np.log(softmax(np.array(scores) / 0.75)))

            for word, score in zip(words, scores):
                self.candidates[var]["weights"][word] = score

            for word, score in zip(words, scores):
                self.candidates[var]["weights"][word] = score
            weights = self.candidates[var]["weights"]
            self.candidates[var]["words"] = sorted(weights, key=weights.get)
            self.candidates[var]["bit_array"] = np.zeros((len(chars), length, len(self.candidates[var]["words"])))
            self.candidates[var]["single_query_cache"] = [defaultdict(lambda:[]) for _ in range(len(chars))]
            self.candidates[var]["single_query_cache_indices"] = [defaultdict(lambda:[]) for _ in range(len(chars))]
            for word_idx, word in enumerate(self.candidates[var]["words"]):
                for pos_idx, char in enumerate(word):
                    char_idx = self.char_map[char]
                    self.candidates[var]["bit_array"][char_idx, pos_idx, word_idx] = 1
                    self.candidates[var]["single_query_cache"][pos_idx][char].append(word)
                    self.candidates[var]["single_query_cache_indices"][pos_idx][char].append(word_idx)
                    # NOTE: TODO, it's possible to cache more here in exchange for doing more work at init time

        # cleanup a bit


IndentationError: ignored

In [127]:
class BPSolver(Solver):
    def __init__(self,
                 crossword,
                 max_candidates = 40000,
                 process_id=0,
                 **kwargs):
        super().__init__(crossword,
                         process_id=process_id,
                         **kwargs)
        self.crossword = crossword
        self.reset()

    # the first function after solving the crossword using first pass model, begins with the reset local function
    def reset(self):
        self.bp_cells = []
        self.bp_cells_by_clue = defaultdict(lambda: [])

        # defining every cells (one with letter in it) in the crossword
        # crossword.grid_cells.items() -> (0, 0): ['1A', '1D'], key -> Position, value -> Clue Positional Info

        for position, clue_pair in self.crossword.grid_cells.items():
            cell = BPCell(position, clue_pair) # need to seriously check this
            self.bp_cells.append(cell)
            for clue in clue_pair:
                self.bp_cells_by_clue[clue].append(cell)

        self.bp_vars = []
        for key, value in self.crossword.variables.items():
            var = BPVar(key, value, self.candidates[key], self.bp_cells_by_clue[key])
            self.bp_vars.append(var)

    def extract_float(self, input_string):
        pattern = r"\d+\.\d+"
        matches = re.findall(pattern, input_string)
        float_numbers = [float(match) for match in matches]
        return float_numbers

    def solve(self, num_iters = 10, iterative_improvement_steps = 5, return_greedy_states = False, return_ii_states = False):
        output_results = {}
        # run solving for num_iters iterations
        print('\nBeginning Belief Propagation iteration steps')
        for _ in tqdm(range(num_iters), ncols = 100):
            for var in self.bp_vars:
                var.propagate()
            for cell in self.bp_cells:
                cell.sync_state()
            for cell in self.bp_cells:
                cell.propagate()
            for var in self.bp_vars:
                var.sync_state()
        print('Belief Propagation iteration complete\n')

        # Get the current based grid based on greedy selection from the marginals
        if return_greedy_states:
            grid, all_grids = self.greedy_sequential_word_solution(return_grids = True)
        else:
            grid = self.greedy_sequential_word_solution()
            all_grids = []

        return grid

    def greedy_sequential_word_solution(self, return_grids = False):
        all_grids = []
        cache = [(deepcopy(var.words), deepcopy(var.log_probs)) for var in self.bp_vars]

        grid = [["" for _ in row] for row in self.crossword.letter_grid]
        unfilled_cells = set([cell.position for cell in self.bp_cells])
        for var in self.bp_vars:
            # postprocess log probs to estimate probability that you don't have the right candidate
            var.log_probs = var.log_probs + math.log(1 - LETTER_SMOOTHING_FACTOR[var.length])
        best_per_var = [var.log_probs.max() for var in self.bp_vars]
        while not all([x is None for x in best_per_var]):
            all_grids.append(deepcopy(grid))
            best_index = best_per_var.index(max([x for x in best_per_var if x is not None]))
            best_var = self.bp_vars[best_index]
            best_word = best_var.words[best_var.log_probs.argmax()]
            # print('greedy filling in', best_word)
            for i, cell in enumerate(best_var.ordered_cells):
                letter = best_word[i]
                grid[cell.position[0]][cell.position[1]] = letter
                if cell.position in unfilled_cells:
                    unfilled_cells.remove(cell.position)
                for var in cell.crossing_vars:
                    if var != best_var:
                        cell_index = var.ordered_cells.index(cell)
                        keep_indices = [j for j in range(len(var.words)) if var.words[j][cell_index] == letter]
                        var.words = [var.words[j] for j in keep_indices]
                        var.log_probs = var.log_probs[keep_indices]
                        var_index = self.bp_vars.index(var)
                        if len(keep_indices) > 0:
                            best_per_var[var_index] = var.log_probs.max()
                        else:
                            best_per_var[var_index] = None
            best_var.words = []
            best_var.log_probs = best_var.log_probs[[]]
            best_per_var[best_index] = None

        for cell in self.bp_cells:
            if cell.position in unfilled_cells:
                grid[cell.position[0]][cell.position[1]] = string.ascii_uppercase[cell.log_probs.argmax()]
                '''
                some alternative improvement can be done at this point, i think so
                instead of filling the cell with highest valued letter, what if we find the associated variables or filling with the cell, and insert letter such that the joint probability of occuring the answers, or letters in all the associated filling is maximum
                '''

        for var, (words, log_probs) in zip(self.bp_vars, cache): # restore state
            var.words = words
            var.log_probs = log_probs
        if return_grids:
            return grid, all_grids
        else:
            return grid

In [134]:
MAIN_DATASET = "/content/gdrive/MyDrive/Clue-Answer Dataset/CLUE_ANSWER_DATA_VERSION_1.csv"

main_df = pd.read_csv(MAIN_DATASET)
main_df = main_df[['clue', 'answer']]
main_df['answer'] = main_df['answer'].apply(lambda x: x.replace(' ', '').upper() if not isinstance(x, float) else "PRAJESH-INVALID")
main_df = main_df[~(main_df['answer'] == 'PRAJESH-INVALID')]
len(main_df), main_df.head()

(6771138,
                                 clue  answer
 0     cpa for example crossword clue  TAXPRO
 1             cordial crossword clue    WARM
 2    joe without jolt crossword clue   DECAF
 3   bouillabaisse bit crossword clue  MUSSEL
 4  pre-gps travel aid crossword clue  AAAMAP)

In [137]:
sampled_df = main_df[main_df['answer'].apply(len) == 4].sample(n = 1000)
sampled_df['answer'].tolist()

['FROM',
 'WEAN',
 'TIES',
 'IDEM',
 'DRUG',
 'KNEW',
 'WIFI',
 'POET',
 'EYES',
 'TWOS',
 'CASH',
 'EVES',
 'CASE',
 'WARD',
 'ALAI',
 'OBIS',
 'EVA,',
 'AMMO',
 'NATE',
 'BEAN',
 'YENS',
 'URDU',
 'NOIR',
 'SARI',
 'AVID',
 'ARLO',
 'VEST',
 'ASTA',
 'CAEN',
 'MADE',
 'ROTO',
 'RAGE',
 'OCRA',
 'INC,',
 'FINE',
 'SLAW',
 'UHUH',
 'AMIS',
 'SCAB',
 'LACE',
 'CHIA',
 'APRS',
 'AGUA',
 'STLO',
 'CONE',
 'SLAV',
 'GOOF',
 'DOAK',
 'LENA',
 'BOND',
 'GUAC',
 'ONIT',
 'CRAN',
 'MAYO',
 'CHUG',
 'FRAN',
 'LENS',
 'DOCK',
 'TEST',
 'HURT',
 'ESPN',
 'OSLO',
 'TRET',
 'FLOE',
 'BREE',
 'HART',
 'ADAM',
 'RIDE',
 'ASHE',
 'ISLE',
 'DOES',
 'TIME',
 'ADAM',
 'AOKS',
 'EINS',
 'VDAY',
 'BALI',
 'CHIA',
 'OKIE',
 'LEIS',
 'ESPY',
 'HUBS',
 'AVIA',
 'AERO',
 'OWLS',
 'SISI',
 'BRAD',
 'LAST',
 'TASK',
 'ANOD',
 'SIRE',
 'TEAM',
 'MIDI',
 'BURY',
 'SLED',
 'TEAR',
 'MCLI',
 'SOAP',
 'ALLY',
 'TAME',
 'STEM',
 'HIFI',
 'MOLD',
 'EMAJ',
 'EDEN',
 'IFFY',
 'WOLF',
 'TEST',
 'IANS',
 'ATTU',
 'BYTE',
 

In [138]:
import random

def assign_random_values(strings):
    result = {}
    for string in strings:
        result[string] = random.uniform(40, 60)
    return result

# Example usage:
string_list = ["string1", "string2", "string3"]
random_values = assign_random_values(string_list)

for string, value in random_values.items():
    print(f"{string}: {value}")


string1: 51.7812382948855
string2: 58.70916501631539
string3: 42.893504867884616
